In [3]:
#import packages
import os
import pandas as pd
import numpy as np
df = pd.read_csv('bank/bank.csv',sep=';') #Read excel file
#df = pd.read_excel('bank.xlsx') #Read excel file

In [4]:
df.head() 

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null object
dtypes: int64(7), object(10)
memory usage: 600.5+ KB


In [6]:
df['y'].value_counts()

no     4000
yes     521
Name: y, dtype: int64

In [7]:
df['target'] = df['y'].apply(lambda x : 1 if x == 'yes' else 0)  # Convert to numeric
df = df.drop('y',axis=1)

In [8]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,target
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


# WOE and IV

In [59]:
stats.spearmanr?

In [9]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin): #连续型
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0  # 单调相关性
    while np.abs(r) < 1: #初始值为0,所以一定会进入此分支, 为何知道r=1  会成立呢? 或者是  nan也会跳出循环, n=1时,必定是nan
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)  # 将x分桶然后计算分桶x与y之间的相关性
            # 单调分桶可确保自变量和因变量之间建立线性关系
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y) # 返回单调相关性以及p值
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1: #如果只产生了一个group,那么此处强制产生force_bin个group
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y #　此处为所有事件发生的次数
    d3["EVENT"] = d2.sum().Y # 这个为１的事件发生的次数
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y # 这个为0的事件发生的次数
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0: # 此处的miss,也只是x有miss,但y没有miss,否则样本没有意义
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT #与d3["EVENT"].sum()等价
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT) # 按公式计算
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()  # 将各个bin的iv进行汇总
    
    return(d3)

def char_bin(Y, X): # 离散型
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)  # 按照x的取值进行聚类分组
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum() # 对各个分段内的求和
    d3 = d3.reset_index(drop=True) # 去掉index
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index # 
    count = -1
    
    for i in x: # 遍历列名
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2: # 连续型变量
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else: # 离散型变量
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True) # 按行append
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()}) # 返回每个feature的iv值, var_name会自动保留
    iv = iv.reset_index()
    return(iv_df,iv) 

In [100]:
Y=df["target"]
X=df["balance"]
n = max_bin

In [101]:
X.head()

0    1787
1    4789
2    1350
3    1476
4       0
Name: balance, dtype: int64

In [102]:
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0  # 单调相关性
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)}) # n会影响qcut的个数
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y) # 返回单调相关性以及p值
            n = n - 1 # bin的个数减1
        except Exception as e:
            n = n - 1

In [103]:
notmiss["X"].head()

0    1787
1    4789
2    1350
3    1476
4       0
Name: X, dtype: int64

In [104]:
pd.qcut(notmiss.X, n).head()

0    (979.333, 71188.0]
1    (979.333, 71188.0]
2    (979.333, 71188.0]
3    (979.333, 71188.0]
4    (-3313.001, 174.0]
Name: X, dtype: category
Categories (3, interval[float64]): [(-3313.001, 174.0] < (174.0, 979.333] < (979.333, 71188.0]]

In [123]:
d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, 2)})
d1.head()

,Bucket,X,Y
0,"(444.0, 71188.0]",1787,0
1,"(444.0, 71188.0]",4789,0
2,"(444.0, 71188.0]",1350,0
3,"(444.0, 71188.0]",1476,0
4,"(-3313.001, 444.0]",0,0


In [124]:
d2 = d1.groupby('Bucket', as_index=True)
d2.head(5)    

,Bucket,X,Y
0,"(444.0, 71188.0]",1787,0
1,"(444.0, 71188.0]",4789,0
2,"(444.0, 71188.0]",1350,0
3,"(444.0, 71188.0]",1476,0
4,"(-3313.001, 444.0]",0,0
5,"(444.0, 71188.0]",747,0
6,"(-3313.001, 444.0]",307,0
7,"(-3313.001, 444.0]",147,0
8,"(-3313.001, 444.0]",221,0
9,"(-3313.001, 444.0]",-88,0


In [125]:
d2.mean() # 连续值按分位点分为3段,每段计算均值, 以bucket为index

,X,Y
Bucket,,
"(-3313.001, 444.0]",71.210526,0.088456
"(444.0, 71188.0]",2774.703097,0.142035


In [83]:
d2.mean().X

Bucket
(-3313.001, 174.0]     -41.649471
(174.0, 979.333]       489.847537
(979.333, 71188.0]    3821.538819
Name: X, dtype: float64

In [107]:
d2.mean()["X"]

Bucket
(-3313.001, 174.0]     -41.649471
(174.0, 979.333]       489.847537
(979.333, 71188.0]    3821.538819
Name: X, dtype: float64

In [108]:
d2.min()["X"] #在group index的组上计算最小值

Bucket
(-3313.001, 174.0]   -3313
(174.0, 979.333]       175
(979.333, 71188.0]     980
Name: X, dtype: int64

In [126]:
r, p = stats.spearmanr(d2.mean().X, d2.mean().Y) # 计算x,y上的相关性

In [127]:
r,p

(0.9999999999999999, nan)

In [134]:
np.nan <1

False

In [98]:
d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, 3)})
d1.head()
d2 = d1.groupby('Bucket', as_index=True)
d2.head(5)    
r, p = stats.spearmanr(d2.mean().X, d2.mean().Y) # 计算x,y上的相关性
r,p

(1.0, 0.0)

In [109]:
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y #　此处为所有事件发生的次数
    d3["EVENT"] = d2.sum().Y # 这个为１的事件发生的次数
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y # 这个为0的事件发生的次数
    d3=d3.reset_index(drop=True)
    

In [110]:
d3

,MIN_VALUE,MAX_VALUE,COUNT,EVENT,NONEVENT
0,-3313,174,1512,133,1379
1,175,979,1502,163,1339
2,980,71188,1507,225,1282


In [111]:
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)

In [112]:
d3

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,VAR,-3313,174,1512,133,0.087963,1379,0.912037,0.255278,0.34475,-0.300465,0.026883
1,VAR,175,979,1502,163,0.108522,1339,0.891478,0.312860,0.33475,-0.067629,0.001480
2,VAR,980,71188,1507,225,0.149303,1282,0.850697,0.431862,0.32050,0.298223,0.033211


In [115]:
 d3.IV.sum()

0.18472263908552927

In [129]:
 d3["IV"].sum()

0.18472263908552927

In [128]:
 d3.sum().IV

0.18472263908552927

In [113]:
d3.IV = d3.IV.sum()
#d3 = d3.reset_index(drop=True)

In [130]:
 d3.IV = d3.IV.sum() # 对各个分段内的求和

In [132]:
d3.head()

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,VAR,-3313,174,1512,133,0.087963,1379,0.912037,0.255278,0.34475,-0.300465,0.184723
1,VAR,175,979,1502,163,0.108522,1339,0.891478,0.312860,0.33475,-0.067629,0.184723
2,VAR,980,71188,1507,225,0.149303,1282,0.850697,0.431862,0.32050,0.298223,0.184723


In [10]:
final_iv, IV = data_vars(df,df.target)

In [133]:
final_iv.head()

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,age,19,39,2290,259,0.113100,2031,0.886900,0.497121,0.50775,-0.021156,0.000452
1,age,40,87,2231,262,0.117436,1969,0.882564,0.502879,0.49225,0.021363,0.000452
2,job,admin.,admin.,478,58,0.121339,420,0.878661,0.111324,0.10500,0.058488,0.132519
3,job,blue-collar,blue-collar,946,69,0.072939,877,0.927061,0.132438,0.21925,-0.504101,0.132519
4,job,entrepreneur,entrepreneur,168,15,0.089286,153,0.910714,0.028791,0.03825,-0.284088,0.132519


In [12]:
IV.sort_values('IV')

,VAR_NAME,IV
5,default,0.000016
0,age,0.000452
4,day,0.004581
2,campaign,0.023342
7,education,0.031812
11,marital,0.040090
10,loan,0.060791
1,balance,0.076208
8,housing,0.106556
9,job,0.132519


In [13]:
IV.to_csv('test.csv')

# Apply WOE values to your dataframe columns

The below code snippet can be used to apply the WOE values to your dataframe columns.

In [21]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'target'],
      dtype='object')

In [22]:
df.columns.difference(['target'])

Index(['age', 'balance', 'campaign', 'contact', 'day', 'default', 'duration',
       'education', 'housing', 'job', 'loan', 'marital', 'month', 'pdays',
       'poutcome', 'previous'],
      dtype='object')

In [14]:
transform_vars_list = df.columns.difference(['target'])
transform_prefix = 'new_' # leave this value blank if you need replace the original column values

In [15]:
transform_vars_list

Index(['age', 'balance', 'campaign', 'contact', 'day', 'default', 'duration',
       'education', 'housing', 'job', 'loan', 'marital', 'month', 'pdays',
       'poutcome', 'previous'],
      dtype='object')

In [36]:
small_df = final_iv[final_iv['VAR_NAME'] == 'contact']

In [37]:
small_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
31,contact,cellular,cellular,2896,416,0.143646,2480,0.856354,0.798464,0.62000,0.252971,0.247762
32,contact,telephone,telephone,301,44,0.146179,257,0.853821,0.084453,0.06425,0.273413,0.247762
33,contact,unknown,unknown,1324,61,0.046073,1263,0.953927,0.117083,0.31575,-0.992072,0.247762


In [39]:
transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
transform_dict

{'cellular': 0.2529710194508961,
 'telephone': 0.273413147371702,
 'unknown': -0.992071659828519}

In [46]:
sorted(transform_dict.items()) # 按key的字母顺序排列

[('cellular', 0.2529710194508961),
 ('telephone', 0.273413147371702),
 ('unknown', -0.992071659828519)]

In [42]:
replace_cmd = ''
replace_cmd1 = ''
for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
replace_cmd = replace_cmd + '0'
replace_cmd1 = replace_cmd1 + '0'

In [57]:
[i for i in range(5)
 for j in range(i )]

[1, 2, 2, 3, 3, 3, 4, 4, 4, 4]

In [56]:
[i for i in range(5) 
 for j in range(i+1 )]

[0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4]

In [43]:
replace_cmd

'0.2529710194508961 if x <= cellular else 0.273413147371702 if x <= telephone else -0.992071659828519 if x <= unknown else 0'

In [58]:
replace_cmd1 #多个if解析 --- 先比较前面,然后比较后面

'0.2529710194508961 if x == "cellular" else 0.273413147371702 if x == "telephone" else -0.992071659828519 if x == "unknown" else 0'

In [49]:
func=lambda x: eval(replace_cmd1)

In [50]:
func("unknown")

-0.992071659828519

In [51]:
func("telephone")

0.273413147371702

In [52]:
func("cellular")

0.2529710194508961

In [23]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd != '0':
        try:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd)) # df["new_duration"] = woe值
        except:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd1))

In [28]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,new_duration,new_education,new_housing,new_job,new_loan,new_marital,new_month,new_pdays,new_poutcome,new_previous
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,...,-1.857594,-0.222812,0.330235,-0.141683,0.090598,-0.169697,1.888017,-0.263692,-0.263692,-0.190217
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,...,0.069240,-0.091389,-0.325552,-0.261650,-0.674391,-0.169697,-0.603059,0.793417,0.124650,0.944712
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,...,-0.875304,0.247404,-0.325552,0.182479,0.090598,0.219951,0.595591,0.793417,0.124650,-0.190217
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,...,-0.875304,0.247404,-0.325552,0.182479,-0.674391,-0.169697,-0.119785,-0.263692,-0.263692,-0.190217
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,...,0.069240,-0.091389,-0.325552,-0.504101,0.090598,-0.169697,-0.603059,-0.263692,-0.263692,-0.190217


In [24]:
df['contact'].value_counts()

cellular     2896
unknown      1324
telephone     301
Name: contact, dtype: int64

In [29]:
df['new_contact'].value_counts()  # 可以看到,它们是以woe为值

 0.252971    2896
-0.992072    1324
 0.273413     301
Name: new_contact, dtype: int64

In [26]:
small_df = final_iv[final_iv['VAR_NAME'] == 'contact']

In [27]:
small_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
31,contact,cellular,cellular,2896,416,0.143646,2480,0.856354,0.798464,0.62000,0.252971,0.247762
32,contact,telephone,telephone,301,44,0.146179,257,0.853821,0.084453,0.06425,0.273413,0.247762
33,contact,unknown,unknown,1324,61,0.046073,1263,0.953927,0.117083,0.31575,-0.992072,0.247762
